# 3. InceptionResNetV2_SE_Predict
Kaggle score: 0.000

## Run name

In [1]:
import time

project_name = 'inclusive-images-challenge'
step_name = '3_InceptionResNetV2_SE_Predict'
time_str = time.strftime("%Y%m%d-%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)

time0 = time.time()

run_name: inclusive-images-challenge_3_InceptionResNetV2_SE_Predict_20181028-130513


## Important params

In [2]:
debug_rows = None #100*10000
image_size = 200
batch_size = 1024
model_name0 = 'inclusive-images-challenge_3_InceptionResNetV2_SE_20181027-030909_2301'
model_name = '%s.h5' % model_name0
tag_info_name = 'inclusive-images-challenge_3_InceptionResNetV2_SE_20181027-030909_tag_info.pkl'

print(debug_rows, batch_size)
print(model_name)
print(tag_info_name)

None 1024
inclusive-images-challenge_3_InceptionResNetV2_SE_20181027-030909_2301.h5
inclusive-images-challenge_3_InceptionResNetV2_SE_20181027-030909_tag_info.pkl


## Import pkgs

In [3]:
import os
import random
import time
import re
import pickle
from tqdm import tqdm
import multiprocessing
from tqdm import tqdm_notebook
import imghdr

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score, jaccard_similarity_score

cpu_amount = multiprocessing.cpu_count()
random_num = np.random.randint(0, 9999)
random_num_str = '%04d' % random_num

print('cpu_amount: %s' % (cpu_amount - 1))
print('random_num: %s' % random_num_str)

cpu_amount: 3
random_num: 0566


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Project folders

In [4]:
cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
log_folder = os.path.join(cwd, 'log')

cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
log_folder = os.path.join(cwd, 'log')

print(feature_folder)
print(input_folder)
print(output_folder)
print(model_folder)
print(log_folder)
train_folder = os.path.join(input_folder, 'train')
validation_folder = os.path.join(input_folder, 'validation')
stage_1_test_images_folder = os.path.join(input_folder, 'stage_1_test_images')
sl_train_folder = os.path.join(input_folder, 'sl_train')
sl_val_folder = os.path.join(input_folder, 'sl_val')
sl_test_folder = os.path.join(input_folder, 'sl_test')

print(train_folder)
print(validation_folder)
print(stage_1_test_images_folder)
print(sl_train_folder)
print(sl_val_folder)
print(sl_test_folder)

class_descriptions_csv = os.path.join(input_folder, 'class-descriptions.csv')
class_trainable_csv = os.path.join(input_folder, 'classes-trainable.csv')
stage_1_attributions_csv = os.path.join(input_folder, 'stage_1_attributions.csv')
stage_1_sample_submission_csv = os.path.join(input_folder, 'stage_1_sample_submission.csv')
train_bounding_boxes_csv = os.path.join(input_folder, 'train_bounding_boxes.csv')
train_human_labels_csv = os.path.join(input_folder, 'train_human_labels.csv')
train_machine_labels_csv = os.path.join(input_folder, 'train_machine_labels.csv')
tuning_labels_csv = os.path.join(input_folder, 'tuning_labels.csv')

print(class_descriptions_csv)
print(class_trainable_csv)
print(stage_1_attributions_csv)
print(stage_1_sample_submission_csv)
print(train_bounding_boxes_csv)
print(train_human_labels_csv)
print(train_machine_labels_csv)
print(tuning_labels_csv)

/data/kaggle-Inclusive-Images/feature
/data/kaggle-Inclusive-Images/input
/data/kaggle-Inclusive-Images/output
/data/kaggle-Inclusive-Images/model
/data/kaggle-Inclusive-Images/log
/data/kaggle-Inclusive-Images/input/train
/data/kaggle-Inclusive-Images/input/validation
/data/kaggle-Inclusive-Images/input/stage_1_test_images
/data/kaggle-Inclusive-Images/input/sl_train
/data/kaggle-Inclusive-Images/input/sl_val
/data/kaggle-Inclusive-Images/input/sl_test
/data/kaggle-Inclusive-Images/input/class-descriptions.csv
/data/kaggle-Inclusive-Images/input/classes-trainable.csv
/data/kaggle-Inclusive-Images/input/stage_1_attributions.csv
/data/kaggle-Inclusive-Images/input/stage_1_sample_submission.csv
/data/kaggle-Inclusive-Images/input/train_bounding_boxes.csv
/data/kaggle-Inclusive-Images/input/train_human_labels.csv
/data/kaggle-Inclusive-Images/input/train_machine_labels.csv
/data/kaggle-Inclusive-Images/input/tuning_labels.csv


## Functions

In [5]:
def pickle_dump(obj, file):
    with open(file, 'wb') as f:
        pickle.dump(obj, f)
        
def pickle_load(file):
    with open(file, 'rb') as f:
        obj = pickle.load(f)
        return obj

## Load csv

In [6]:
%%time
train_bounding_boxes_df = pd.read_csv(train_bounding_boxes_csv, nrows=debug_rows)

CPU times: user 18.3 s, sys: 1.36 s, total: 19.6 s
Wall time: 19.7 s


In [7]:
%%time
stage_1_attributions_df = pd.read_csv(stage_1_attributions_csv)

CPU times: user 28 ms, sys: 0 ns, total: 28 ms
Wall time: 26.7 ms


In [8]:
%%time
stage_1_sample_submission_df = pd.read_csv(stage_1_sample_submission_csv)

CPU times: user 28 ms, sys: 0 ns, total: 28 ms
Wall time: 25.9 ms


In [9]:
display('train_bounding_boxes_df', train_bounding_boxes_df.shape, train_bounding_boxes_df.head(2))
# display('train_machine_labels_df', train_machine_labels_df.shape, train_machine_labels_df.head(2))
# display('train_human_labels_df', train_human_labels_df.shape, train_human_labels_df.head(2))
display('stage_1_attributions_df', stage_1_attributions_df.shape, stage_1_attributions_df.head(2))
display('stage_1_sample_submission_df', stage_1_sample_submission_df.shape, stage_1_sample_submission_df.head(2))

'train_bounding_boxes_df'

(14610229, 13)

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,000002b66c9c498e,xclick,/m/01g317,1,0.0125,0.195312,0.148438,0.587500,0,1,0,0,0
1,000002b66c9c498e,xclick,/m/01g317,1,0.0250,0.276563,0.714063,0.948438,0,1,0,0,0


'stage_1_attributions_df'

(32580, 2)

,image_id,source
0,2b2b327132556c767a736b3d,Nita Ar
1,2b2b394755692f303963553d,Sathya Esarapu


'stage_1_sample_submission_df'

(32580, 2)

,image_id,labels
0,2b2b327132556c767a736b3d,/m/0sgh53y /m/0g4cd0
1,2b2b394755692f303963553d,/m/0sgh53y /m/0g4cd0


In [10]:
print(len(set(train_bounding_boxes_df['LabelName'])), '--> Use this 599 labels, approximate 600 labels')
# print(len(set(train_machine_labels_df['LabelName'])))
# print(len(set(train_human_labels_df['LabelName'])))
# print(len(set(stage_1_sample_submission_df['labels'])))

599 --> Use this 599 labels, approximate 600 labels


## Load model

In [11]:
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, TensorBoard
from keras.utils import Sequence
from keras import backend as K

from skimage.io import imread
from skimage.transform import resize

Using TensorFlow backend.


In [12]:
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(
    sl_test_folder, 
    target_size=(image_size, image_size),
    batch_size=batch_size,
    shuffle=False,
    class_mode='categorical'
)
display(len(test_generator.filenames), type(test_generator.filenames), test_generator.filenames[:5])
test_steps = int(len(test_generator.filenames) / batch_size) + 1
print(test_steps)

Found 32536 images belonging to 1 classes.


32536

list

['test/2b2b327132556c767a736b3d.jpg',
 'test/2b2b394755692f303963553d.jpg',
 'test/2b2b42584e6d445937444d3d.jpg',
 'test/2b2b44744e57674270616f3d.jpg',
 'test/2b2b4b425a592b683059493d.jpg']

32


In [13]:
model_file = os.path.join(model_folder, model_name)
print(model_file)
tag_info_file = os.path.join(model_folder, tag_info_name)
print(tag_info_file)

/data/kaggle-Inclusive-Images/model/inclusive-images-challenge_3_InceptionResNetV2_SE_20181027-030909_2301.h5
/data/kaggle-Inclusive-Images/model/inclusive-images-challenge_3_InceptionResNetV2_SE_20181027-030909_tag_info.pkl


In [14]:
model = load_model(model_file)
print(type(model))

<class 'keras.engine.training.Model'>


In [15]:
tag_info = pickle_load(tag_info_file)
print(tag_info.keys())

dict_keys(['train_class_indices', 'val_class_indices', 'test_class_indices'])


In [16]:
label2num = tag_info['train_class_indices']
num2label = {}
for key in label2num:
    num2label[label2num[key]] = key
print(num2label)

{0: '-m-011k07', 1: '-m-012074', 2: '-m-0120dh', 3: '-m-01226z', 4: '-m-012n7d', 5: '-m-012w5l', 6: '-m-012xff', 7: '-m-012ysf', 8: '-m-0130jx', 9: '-m-0138tl', 10: '-m-013y1f', 11: '-m-01432t', 12: '-m-014j1m', 13: '-m-014sv8', 14: '-m-014trl', 15: '-m-014y4n', 16: '-m-0152hh', 17: '-m-01599', 18: '-m-015h_t', 19: '-m-015p6', 20: '-m-015qbp', 21: '-m-015qff', 22: '-m-015wgc', 23: '-m-015x4r', 24: '-m-015x5n', 25: '-m-0162_1', 26: '-m-0167gd', 27: '-m-016m2d', 28: '-m-0174k2', 29: '-m-0174n1', 30: '-m-0175cv', 31: '-m-0176mf', 32: '-m-017ftj', 33: '-m-018j2', 34: '-m-018p4k', 35: '-m-018xm', 36: '-m-01940j', 37: '-m-0199g', 38: '-m-019dx1', 39: '-m-019h78', 40: '-m-019jd', 41: '-m-019w40', 42: '-m-01_5g', 43: '-m-01_bhs', 44: '-m-01b638', 45: '-m-01b7fy', 46: '-m-01b9xk', 47: '-m-01bfm9', 48: '-m-01bjv', 49: '-m-01bl7v', 50: '-m-01bms0', 51: '-m-01bqk0', 52: '-m-01btn', 53: '-m-01c648', 54: '-m-01cmb2', 55: '-m-01d380', 56: '-m-01d40f', 57: '-m-01dws', 58: '-m-01dwsz', 59: '-m-01dwwc',

In [17]:
test_result = model.predict_generator(test_generator, steps=test_steps, max_queue_size=batch_size, workers=1, use_multiprocessing=False, verbose=1)
print(type(test_result), test_result.shape)

32/32 [==============================] - 376s 12s/step
<class 'numpy.ndarray'> (32536, 599)


In [18]:
test_result_sort = np.argsort(test_result)
display(test_result_sort[:5, :20])

array([[472, 393, 503, 382, 264, 453,  76,  11, 263,  22, 269, 246, 199,
         59, 361, 577, 413, 535,  50, 206],
       [ 86, 563, 413, 313, 264, 597, 491, 158, 474, 571, 472, 585, 379,
        193, 403, 548, 511, 194,  24, 174],
       [ 22,  76, 269, 472,  63, 503, 382, 306, 393, 264, 199, 535,  59,
        206, 361, 369,  50, 577, 263, 453],
       [ 22, 577, 535, 269,  93, 369, 453, 399, 472,  50,  63, 185,  76,
        263, 207, 264, 503,  83, 306,  59],
       [393, 585, 535, 193, 361, 164, 382, 472, 253, 313, 571,  76, 155,
        511, 264, 597, 333, 554, 473, 548]])

In [19]:
def fetch_max(nparr, filenames, num2label):
    nparr_sort = np.argsort(nparr)
    print(type(nparr_sort))
    print(nparr_sort.shape)
    
    result_arr = []
    temp_df = pd.DataFrame(columns=['image_id', 'labels'])
    idx = 0
    for i, item in enumerate(zip(filenames, nparr_sort, nparr)):
#         if i > 3:
#             break
        image_id = item[0][5:-4]
#         print(image_id)
        labels = [num2label[label] for label in item[1][:2]]
        temp_df.loc[idx] = (image_id, ' '.join(labels))
        idx += 1
        if idx == 1000:
            print('-', end='')
            result_arr.append(temp_df)
            idx = 0
            temp_df = pd.DataFrame(columns=['image_id', 'labels'])
    result_arr.append(temp_df)
    result_df = pd.concat(result_arr)
    return result_df
    
top_n_df = fetch_max(test_result, test_generator.filenames, num2label)
display(top_n_df.shape)
display(top_n_df.head())

<class 'numpy.ndarray'>
(32536, 599)
--------------------------------

(32536, 2)

,image_id,labels
0,2b2b327132556c767a736b3d,-m-0cqn2 -m-07mcwg
1,2b2b394755692f303963553d,-m-01krhy -m-0jwn_
2,2b2b42584e6d445937444d3d,-m-015wgc -m-01j3zr
3,2b2b44744e57674270616f3d,-m-015wgc -m-0l3ms
4,2b2b4b425a592b683059493d,-m-07mcwg -m-0mw_6


In [20]:
merged_df = pd.merge(left=stage_1_sample_submission_df, right=top_n_df, how='left', on=['image_id'])
merged_df = merged_df.drop(columns=['labels_x'])
merged_df.columns = ['image_id', 'labels']
display(merged_df.shape)
display(merged_df.head())

(32580, 2)

,image_id,labels
0,2b2b327132556c767a736b3d,-m-0cqn2 -m-07mcwg
1,2b2b394755692f303963553d,-m-01krhy -m-0jwn_
2,2b2b42584e6d445937444d3d,-m-015wgc -m-01j3zr
3,2b2b44744e57674270616f3d,-m-015wgc -m-0l3ms
4,2b2b4b425a592b683059493d,-m-07mcwg -m-0mw_6


In [21]:
submission_csv_file = os.path.join(output_folder, '%s.csv' % model_name0)
print(submission_csv_file)
merged_df.to_csv(submission_csv_file, index=False)

/data/kaggle-Inclusive-Images/output/inclusive-images-challenge_3_InceptionResNetV2_SE_20181027-030909_2301.csv


In [22]:
print(run_name)
print('Time elapsed: %.2f s' % (time.time() - time0))

inclusive-images-challenge_3_InceptionResNetV2_SE_Predict_20181028-130513
Time elapsed: 543.85 s
